In [1]:
from heuristique.genetique import genese, evaluation, selection
from objects.population import Population
from script.utils import sort_task
from objects.instance import Instance
from script.affichage import plot_gantt, net_voisinage
from script.data import loader 
import numpy as np
import networkx as nx
import copy
import matplotlib.pyplot as plt
import random
from heuristique.heuristique_gloutonne import heuristique_gloutone
from heuristique.ordo_avec_liste import alloc_avec_liste, allocation_dispomachine_ASAP,decodage_bier
from script.utils import vecteur_bier
from script.recherche_voisin import permutation_random

data, optimum= loader('ft06')

In [2]:
pop = Population()
genese(10, data, pop, data)
evaluation(pop, data)
a = selection(pop,2)

0
1
2
3
4
5
6
7
8
9


In [3]:
inst = Instance(**data)
heuristique_gloutone(inst, verbose=0, prio="SPT", rnd=0)
liste = vecteur_bier(inst)



sequence = permutation_random(liste, 10)

In [12]:
for s in sequence:
    print(s)


[4 4 4 4 3 0 3 1 2 2 2 1 3 5 3 5 1 2 1 0 0 2 4 5 5 5 2 0 1 1 4 0 3 5 3 0]
[2 5 2 0 0 3 5 4 5 4 3 3 2 4 4 2 1 1 1 2 0 1 0 3 0 5 0 1 4 3 2 5 1 3 4 5]
[2 3 1 4 3 0 1 4 2 4 5 4 2 0 4 2 0 5 4 5 2 5 0 2 3 5 3 3 1 1 1 5 3 1 0 0]
[3 1 2 2 1 5 3 5 0 3 3 2 4 4 4 2 1 1 0 3 1 0 0 1 0 5 5 4 2 3 2 4 5 0 4 5]
[1 5 0 0 5 4 0 5 4 2 3 1 2 2 5 4 5 1 3 1 2 0 3 3 1 2 4 4 4 2 0 0 5 3 1 3]
[2 2 1 1 3 2 4 4 0 0 5 3 4 4 3 5 2 5 1 1 2 1 3 0 4 0 5 3 1 5 0 5 3 2 4 0]
[5 1 2 2 0 0 3 3 1 3 2 4 4 4 0 5 5 1 4 4 5 3 5 1 2 2 1 3 0 2 1 5 0 3 0 4]
[5 3 0 4 3 5 4 4 5 4 2 2 2 1 5 5 3 0 0 0 1 1 3 4 0 2 3 5 3 2 2 4 1 1 0 1]
[5 3 3 0 2 5 1 2 1 5 4 2 5 1 0 4 1 3 4 5 0 2 3 3 1 3 1 4 2 5 2 0 0 4 4 0]
[4 3 1 4 1 2 5 3 0 5 2 2 1 3 1 0 1 0 0 3 1 4 0 0 3 5 4 3 5 4 2 4 2 5 5 2]


In [ ]:
inst = Instance(**data)
total = heuristique_gloutone(inst, verbose = 0, prio ="SPT" , rnd = 0)
liste = vecteur_bier(inst, par = "Start")
instance = Instance(**data)
print(liste)

In [59]:
from script.recherche_voisin import exploration_voisinage, inversion_all, permutation_random

b=exploration_voisinage(instance , data)

0
[1 1 1 0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 3 5 2 5 4 4 0 4 2 4 1] a:3 O 66
[0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 1 1 1 1 4 2 4 0 4 4 5 2 5 3] _:539 O 67
1
[1 1 1 0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 3 5 2 5 4 4 0 4 2 4 1] a:3 O 66
[0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 1 1 1 1 4 2 4 0 4 4 5 2 5 3] _:539 O 67


In [4]:
net_voisinage(b)

ValueError: not enough values to unpack (expected 3, got 2)

In [25]:
help(b.edges)

Help on OutMultiEdgeView in module networkx.classes.reportviews object:

class OutMultiEdgeView(OutEdgeView)
 |  OutMultiEdgeView(G)
 |  
 |  A EdgeView class for outward edges of a MultiDiGraph
 |  
 |  Method resolution order:
 |      OutMultiEdgeView
 |      OutEdgeView
 |      collections.abc.Set
 |      collections.abc.Mapping
 |      collections.abc.Collection
 |      collections.abc.Sized
 |      collections.abc.Iterable
 |      collections.abc.Container
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, nbunch=None, data=False, keys=False, default=None)
 |      Call self as a function.
 |  
 |  __contains__(self, e)
 |  
 |  __getitem__(self, e)
 |  
 |  __iter__(self)
 |  
 |  __len__(self)
 |  
 |  data(self, data=True, keys=False, default=None, nbunch=None)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  dataview = <class

In [56]:
edge = {}
for u,v in b.edges():
    pass
u
print(b.edges[(u,v,i)])

NameError: name 'i' is not defined

In [58]:
a= list(b.edges)
for i in a :
    print(i)

('O', '[1 1 1 0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 3 5 2 5 4 4 0 4 2 4 1]', 0)
('O', '[1 1 1 0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 3 5 2 5 4 4 0 4 2 4 1]', 1)
('O', '[1 1 1 0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 3 5 2 5 4 4 0 4 2 4 1]', 2)
('O', '[1 1 1 0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 3 5 2 5 4 4 0 4 2 4 1]', 3)
('O', '[1 1 1 0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 3 5 2 5 4 4 0 4 2 4 1]', 4)
('O', '[1 1 1 0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 3 5 2 5 4 4 0 4 2 4 1]', 5)
('O', '[0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 1 1 1 1 4 2 4 0 4 4 5 2 5 3]', 0)
('O', '[0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 1 1 1 1 4 2 4 0 4 4 5 2 5 3]', 1)
('O', '[0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 1 1 1 1 4 2 4 0 4 4 5 2 5 3]', 2)
('O', '[0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 1 1 1 1 4 2 4 0 4 4 5 2 5 3]', 3)
('O', '[0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 1 1 1 1 4 2 4 0 4 4 5 2 5 3]', 4)
('O', '[0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 1 

In [17]:
b.nodes

NodeView(('O', 90, '[1 1 1 0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 3 5 2 5 4 4 0 4 2 4 1]', '[0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 1 1 1 1 4 2 4 0 4 4 5 2 5 3]'))

In [ ]:
for v,truc,root,make in best:
    b= exploration_10meilleurs_voisinage(v, inst,graph ,truc)
    for v,truc,root,make in b:
        print(v,truc, root)
        graph.add_node(str(v), makespan = make)
        graph.add_edge(root, str(v), chemin = truc)

In [ ]:
d

In [ ]:
graph.edges.data()

In [ ]:
d = {}
for g in graph.edges():
    graph.edges[('O', '[1 3 4 1 2 5 3 5 5 2 0 4 0 3 0 3 1 5 0 3 1 5 5 2 1 2 4 0 0 2 4 4 2 1 4 3]')]

In [ ]:
graph.nodes.data()

In [ ]:
d2 = {}
size = []
for g in graph.nodes():
    d2[g]= graph.nodes[g]['makespan']
    size.append(graph.nodes[g]['makespan'])

In [ ]:
d2 = {}
size = []
for g in graph.nodes():
    d2[g]= graph.nodes[g]['makespan']
    size.append(graph.nodes[g]['makespan'])

In [ ]:
import plotly.graph_objects as go

plt.figure(figsize=(8,8))
pos = nx.spring_layout(graph)
nx.draw(graph, pos,node_labels = d2, node_color=range(graph.number_of_nodes()),
        edge_color=range(graph.number_of_edges()), edge_cmap=plt.cm.viridis,
        node_size=[i for i in size], cmap=plt.cm.viridis)
nx.draw_networkx_labels(graph, pos, d2)
nx.draw_networkx_edge_labels(graph, pos, edge_labels =d,width=1.0, alpha=0.5,font_size =9)
plt.show()

In [ ]:
graph = nx.MultiDiGraph()
opti, opti_liste, graph ,opti_inst, change = exploration_voisinage(liste, inst,graph)
for i in range(8):
    if change == "Yes":
        opti, opti_liste, graph ,opti_inst,change = exploration_voisinage(opti_liste, opti_inst,graph)


In [ ]:
import plotly.graph_objects as go
pos = nx.shell_layout(graph)
nx.draw(graph, pos,with_labels =True, node_color=range(graph.number_of_nodes()), node_size=800, cmap=plt.cm.Blues)


In [ ]:
a = np.array([0,1,2,3,4,5]*6)
a = np.sort(a)
a

In [ ]:
instance = Instance(**data)
temp =  alloc_avec_liste(instance,a)
affichage_repetition(instance)
temp

In [ ]:
instance = Instance(**data)
temp =  alloc_avec_liste(instance,a)
j=0
ecart = -1
graph = nx.MultiDiGraph() 
name =str(j)+":"+ str(temp)
graph.add_node(name,makespan=total )

step =0
vect = exploration_voisinage_inversion(a)
it =iter(vect)
verbose = 1
j=1
while ecart < 10:
    instance = Instance(**data)
    l = next(it)
    alloc_avec_liste(instance,l)
    graph.add_node(str(j)+":"+str(instance.makeSpan),makespan=instance.makeSpan) 
    graph.add_edge(name, instance.makeSpan)
    if verbose > 1:
        print("makespan",instance.makeSpan)
    ecart = instance.makeSpan - temp
    
    if verbose > 1:
        print('écart',ecart)
        print('optimum',temp)
        print("====================")

    if instance.makeSpan < temp:
        
        step = 0
        liste = l
        name = str(j)+":"+str(instance.makeSpan)
        j+=1
        i = instance
        temp = instance.makeSpan
        vect = exploration_voisinage_inversion(l)
        it =iter(vect)
    step += 1


In [ ]:
graph.nodes.data()


In [ ]:
graph.nodes.data()
test=[]
d = dict(graph.nodes.data())
for v in d.values():
    print(v)
    for vv in v.values():
        print(vv)
        test.append(vv)


In [ ]:

pos = nx.shell_layout(graph)
d = dict(graph.nodes.data())
nx.draw(graph, pos,with_labels =True, node_color=range(graph.number_of_nodes()),
        edge_color=range(graph.number_of_edges()),
        width=4, edge_cmap=plt.cm.viridis,
        node_size=[i**2 for i in test], cmap=plt.cm.viridis)
plt.show()

In [ ]:
plot_gantt(instance, by="JOB", grouping=False)

In [ ]:
for j in instance.jobs_list:
    print(j)

In [ ]:
liste = affichage_repetition(inst)

In [ ]:
i.makeSpan

In [ ]:
instance =Instance(**data)
alloc_avec_liste(instance,liste)

plot_gantt(instance)


In [ ]:
for j in i.jobs_list:
    print(j)

In [ ]:
liste

In [ ]:
for j in instance.jobs_list:
    print(j)

In [ ]:
plot_gantt(instance, by='MACHINE', grouping =False)

In [ ]:
liste = affichage_repetition(inst)
liste


In [ ]:
liste = affichage_repetition(instance)
liste


In [ ]:
dfs = pd.DataFrame(test)
dfs

In [ ]:
compt = 0
minRND = 100000    
inst2 = Instance(**data)
totalSPT = heuristique_gloutone(inst2, verbose = 0, prio ="SPT" , rnd = 0)
minSPT = totalSPT

for i in range(100):
    inst = Instance(**data)
    totalRND = heuristique_gloutone(inst, verbose = 0, prio ="SPT" , rnd = 1)
    if totalRND < minRND :
        minRND = totalRND
    
    if totalRND > totalSPT:
        compt += 1
        
print("RND perd", compt,"/", len(range(100)))
print("RND gagne", len(range(100)) - compt,"/", len(range(100)))
print("Optimium", optimum)
print("minRND", minRND)
print("minSPT", minSPT)

In [ ]:
plot_gantt(inst2, by= "JOB", grouping = True)

In [ ]:
import numpy as np
a = np.array(a)
b = a[:,0]

In [ ]:
        task_list = sorted(task_list.items(), key=lambda x: x[1][2])

a[:]

In [ ]:
list(b)

In [ ]:
v = [int(x[0][0]) for x in np.array(a)]



In [ ]:
l = affichage_repetition(inst)

In [ ]:
int(True)